In [1]:
from IPython.core.debugger import set_trace
import unittest
tc = unittest.TestCase('__init__')

import numpy as np
# mathjax docs: https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference

# The Problem

Given $A \in \mathbb{R}^{nxn} \text{ and } b \in \mathbb{R}^n$ in the equation:

$Ax = b$
  
We want to solve for $x$ 

In [2]:
A = np.random.random((3,3))
b = np.random.random((3,1))

In [3]:
def test_correct_matrix_solution(A, b, x):
    return np.array_equal(np.dot(A, x), b)

def test_method(method_func):
    # example 6.1.2
    A_1 = np.array([[1, -1, 2 ,-1],
           [2, -2, 3, -3],
           [1, 1, 1, 0],
           [1, -1, 4, 3]])
    b_1 = np.array([[-8, -20, -2, 4]]).T
    x_1 = np.array([2, 2, 3, -7]).T
    tc.assertTrue(np.array_equal(method_func(A_1, b_1), x_1))
    
    # random A & b
    A_2 = np.random.random((4,4))
    b_2 = np.random.random((4,1))
    x_2 = method_func(A_2, b_2)
    tc.assertTrue(np.array_equal(np.dot(A_2, x_2), b_2))

# 1: Gausse Elimination

### No Pivoting:

In [4]:
AC = A.copy() # AC -> Copy of A
AC

array([[0.01289123, 0.84166937, 0.10624828],
       [0.83534194, 0.64911037, 0.92566622],
       [0.43046583, 0.86800027, 0.37422   ]])

In [94]:
def swap_rows(m, row_index_1, row_index_2):
    cpy = m.copy()
    cpy[[row_index_1, row_index_2]] = cpy[[row_index_2, row_index_1]]
    return cpy

def swap_rows_if_needed(m, pivot_row, pivot_col):
        if m[pivot_row, pivot_col] > 0:
            return m
        
        nonzeros = m[pivot_row: m.shape[0], pivot_col].nonzero()
        if len(nonzeros[0]) > 0:
            next_nonzero_index = nonzeros[0][0] + pivot_row
            if next_nonzero_index != pivot_row:
                m = swap_rows(m, pivot_row, next_nonzero_index)

        return m

def swap_rows_if_needed_test():
    A = np.random.random((4,4))
    A[1, 1] = 0
    A_real = swap_rows_if_needed(A, 1, 1)
    A_expected = swap_rows(A, 1, 2)
    tc.assertTrue(np.array_equal(A_expected, A_real))
    
    B = np.random.random((4,4))
    B_real = swap_rows_if_needed(B, 1, 1)
    B_expected = B
    tc.assertTrue(np.array_equal(B_expected, B_real))
    
swap_rows_if_needed_test()

In [101]:
def forward_elimination(m):
    def forward_elimination_recurse(m, pivot_row_index, j):
        m = swap_rows_if_needed(m, pivot_row_index, j)
        pivot_row = m[pivot_row_index]
        
        for row_i in range(pivot_row_index + 1, m.shape[0]):
            calc = pivot_row * np.divide(m[row_i][j], pivot_row, where=pivot_row!=0)
            m[row_i, j] = np.round(np.abs(m[row_i, j]) - np.abs(calc[j]))# round to avoid problem that pivoting solves
            m[row_i+1:, j+1:] = np.abs(m[row_i+1:, j+1:]) - np.abs(calc[j+1:])
        
        if j == m.shape[0]-1 or j >= m.shape[1]-1:
            return m
        else:
            return forward_elimination_recurse(m, pivot_row_index + 1, j + 1)
    
    return forward_elimination_recurse(m, 0, 0)

In [108]:
g = np.random.uniform(0, 5, (4, 4))

print(g)
print("\n")
print(forward_elimination(g))

[[1.1749303  4.28365446 1.09631474 3.07008667]
 [0.99670495 4.85210421 4.56556906 3.6001109 ]
 [0.63179034 2.612491   1.98159778 1.49124098]
 [1.76243941 4.98353836 2.61431596 1.53332818]]


[[ 1.1749303   4.28365446  1.09631474  3.07008667]
 [-0.          4.85210421  4.56556906  3.6001109 ]
 [ 0.          0.          0.98489283  0.49453603]
 [ 0.          0.          0.         -1.52061894]]


In [154]:
t = np.random.random((2,2))
h = np.random.random((2,1))
print(t, "\n\n\n", h)
t * h

[[0.14725548 0.74327655]
 [0.37468017 0.88507139]] 


 [[0.7898082 ]
 [0.55331357]]


array([[0.11630358, 0.58704592],
       [0.20731562, 0.48972201]])

In [155]:
def backward_substitution(m, b):
    '''
    There are only m variables to solve for in an n x m matrix,
    so we can just solve the m x m submatrix. If m > n, we don't
    have enough equations to solve the system.
    

    m = m x m submatrix of input
    b = m x 1 submatrix of input
    v = variable values
    x = output    
    n = # of rows
    k = # of columns
    '''
    m = input_m[-input_m.shape[1]:, :]
    b = input_b[-input_m.shape[1]:]
    v = np.array((input_m.shape[1],))
    
    n = m.shape[0]
    k = m.shape[1]
            
    # first iteration 
    v[k] = b[k] / m[n,k]
    m[0: n-1, k] *= c[k]
    
    rows_range = reversed(range(0, m.shape[1]-1))
    
    '''
    i = row
    j = column
    '''
    for i, j in zip(rows_range, rows_range):
        found_variables_sum = np.sum(m[i, j+1:n])
        v[j] = (b[j] - found_variables_sum) / m[i, j]
        m[0: i-1, j] *= v[j]
    
    if n > k:
        m[0: n-k] *= v
    
    x = np.sum(m, axis=1)
    return x

In [93]:
def no_pivot_gausse(A, b):
    pass